In [1]:
import numpy as np
import pandas as pd
import os, glob, json
from collections import Counter
from nltk.tokenize import wordpunct_tokenize as tokenize
import io, math, time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from gensim.models import word2vec


In [2]:
with open('../data/train/train.json', 'r') as f:
    train = json.load(f)
    
with open('../data/validation/validation.json', 'r') as f:
    validation = json.load(f)
    
with open('../data/vocab/vocab.json', 'r') as f:
    vocab = json.load(f)

In [3]:
word_to_ind = {word:ind for ind, word in enumerate(vocab)}
ind_to_word = {ind:word for ind, word in enumerate(vocab)}

მოდელისთვის გვჭირდება, რომ მთელი კორპუსი ერთ დიდ მიმდევრობად წარმოვადგინოთ. ერთ ლისტში არ გვაკეთებინებდა და ორად დავყავით. 

In [4]:
train_sequence1 = []
for ind, doc in enumerate(train):
    if ind < 25000:
        for word in doc:
            train_sequence1.append(torch.tensor(word_to_ind.get(word, 0)))

train_sequence1 = torch.tensor(train_sequence1)

train_sequence2 = []
for ind, doc in enumerate(train):
    if ind >= 25000:
        for word in doc:
            train_sequence2.append(torch.tensor(word_to_ind.get(word, 0)))

train_sequence2 = torch.tensor(train_sequence2)

In [5]:
valid_sequence = []
for doc in validation:
    for word in doc:
        valid_sequence.append(torch.tensor(word_to_ind.get(word, 0)))

valid_sequence = torch.tensor(valid_sequence)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    nbatch = data.size(0) // bsz
    data = data.narrow(0, 0, nbatch * bsz)
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)


In [7]:
# ბეჩებად დაყოფა
batch_size = 20
eval_batch_size = 10
train_data1 = batchify(train_sequence1, batch_size)
train_data2 = batchify(train_sequence2, batch_size)
val_data = batchify(valid_sequence, eval_batch_size)


In [8]:
# რა სიგრძის მიმდევრობები უნდა გადავცეთ მოდელს
bptt = 30
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target


მოგეხსენებათ attention მექანიზმი არ ითვალისწინებს ტოკენების მიმდევრობას. ეს კი პრობლემაა, რადგან როცა საქმე ტექსტს ეხება, სიტყვათა მიმდევრობა ძალიან მნიშვნელოვანია.

მოცემული პრობლემის მოგვარება შეგვიძლია პოზიციური ენკოდირებით. იდეა იმაშია, რომ ავაგოთ სხვადასხვა სიხშირის სინუსოიდა

![positional encoding](https://image.slidesharecdn.com/paper-review-attention-180607163404/95/attention-is-all-you-need-upc-reading-group-2018-by-santi-pascual-34-638.jpg?cb=1528389383)

მნიშვნელოვანია, რომ იყოს იმავე განზომილების, რაც ჩვენი ვორდ ემბედინგებია, რათა საბოლოოდ შევძლოთ მათი მიმატება.

ბევრნაირი სინუსოიდას საშუალებით, პოზიციურ ენკოდირებას შეუძლია რელაციური პოზიციის ინფორმაციის ჩადება თითოეულ ემბედინგში და ასე ვაგვარებთ ზემოხსენებულ პრობლემას.

In [9]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        """
        სინუსოიდების აგება თავიდან ხდება ერთხელ და შემდეგ გამოთვლილ მნიშვნელობებს ვიყენებთ,
        რათა ემბედინგებს დავუმატოთ attention-მდე.
        აღსანიშნავია, რომ pe, ანუ ჩვენი სინუსოიდების კომბინაცია, არ არის მოდელის პარამეტრი და 
        სწორედ ამიტომაა დამატებული register_buffer მეთოდით, რათა მოდელს არ შეშლოდა პარამეტრში, 
        მაგრამ მაინც სთეითის ნაწილი ყოფილიყო. 
        """
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


რაც შეეხება მოდელს, ვიყენებთ ტრანსფორმერის ორიგინალ არქიტექტურას, რომელიც ნაშრომში - "attention is all you need" იქნა შემოთავაზებული. 

![transformer architecture](https://miro.medium.com/max/856/1*ZCFSvkKtppgew3cc7BIaug.png)



In [10]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5, weights=None):
        super(TransformerModel, self).__init__()
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        if weights is None:
            self.encoder = nn.Embedding(ntoken, ninp)        
        else:
            self.encoder = nn.Embedding.from_pretrained(weights)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output


In [11]:
def get_pretrained_weights(model_path):
    w2v = word2vec.Word2Vec.load(model_path)
    embs = []
    for word in vocab:
        if word in w2v.wv.vocab:
            embs.append(w2v.wv[word])
        else:
            embs.append(np.random.uniform(low=-0.5, high=0.5, size=(w2v.wv.vectors.shape[1],)))
    embs = np.array(embs)
    weights = torch.LongTensor(embs).float()
    return weights

w2v_path = '../input/nlp-word2vec/w2vemb100wind4.model'
weights = get_pretrained_weights(w2v_path)

In [12]:
ntokens = len(word_to_ind) # vocab-ის ზომა
emsize = 100 # ემბედინგის განზომილება
nhid = 200 # ენკოდერში ფიდფორვარდ ნეთვორქის ჰიდენ სთეითის განზომილება
nlayers = 2 # ენკოდერ ლეიერების რაოდენობა
nhead = 2 # attention-ების რაოდენობა
dropout = 0.3 # დროფაუთი
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)


In [13]:
criterion = nn.CrossEntropyLoss()
lr = 1.5
optimizer = torch.optim.Adagrad(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(train_data):
    # თრეინ მოუდი
    model.train() 
    total_loss = 0.
    start_time = time.time()
    # მასქინგი აუცილებელია, რათა დექოდერი ვერ ხედავდეს მომდვნო სიტყვებს
    # კვადრატული კი იმიტომაა, რომ ყოველი ეტაპისთვის იყო დამასკული
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        if data.size(0) != bptt:
            src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        # გრადიენტის ქლიფინგი
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        
        total_loss += loss.item()
        log_interval = 1000
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            # როცა ქროს ენთროფი ლოსს ვიყენებთ, პირდაპირ შეგვიძლია exp-ის მორტყმით
            # გამოვთვალოთ ფერფლექსითი
            perplexity = math.exp(cur_loss)
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, perplexity))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval()
    total_loss = 0.
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            if data.size(0) != bptt:
                src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
            output = eval_model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)


In [14]:
best_val_loss = float("inf")
epochs = 15
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(train_data1)
    train(train_data2)
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    val_perplexity = math.exp(val_loss)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, val_perplexity))
    print('-' * 89)
        
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model
        torch.save(best_model.state_dict(), 'best_model' + str(epoch) + '.pth')

    scheduler.step()


/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:370: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


| epoch   1 |  1000/18571 batches | lr 1.50 | ms/batch 22.68 | loss 13.62 | ppl 818765.65
| epoch   1 |  2000/18571 batches | lr 1.50 | ms/batch 21.85 | loss  7.60 | ppl  1997.08
| epoch   1 |  3000/18571 batches | lr 1.50 | ms/batch 22.14 | loss  7.58 | ppl  1950.00
| epoch   1 |  4000/18571 batches | lr 1.50 | ms/batch 21.90 | loss  7.55 | ppl  1909.09
| epoch   1 |  5000/18571 batches | lr 1.50 | ms/batch 22.05 | loss  7.48 | ppl  1771.51
| epoch   1 |  6000/18571 batches | lr 1.50 | ms/batch 21.97 | loss  7.49 | ppl  1793.80
| epoch   1 |  7000/18571 batches | lr 1.50 | ms/batch 21.84 | loss  7.44 | ppl  1702.05
| epoch   1 |  8000/18571 batches | lr 1.50 | ms/batch 22.27 | loss  7.45 | ppl  1715.17
| epoch   1 |  9000/18571 batches | lr 1.50 | ms/batch 21.79 | loss  7.45 | ppl  1717.83
| epoch   1 | 10000/18571 batches | lr 1.50 | ms/batch 21.80 | loss  7.43 | ppl  1679.42
| epoch   1 | 11000/18571 batches | lr 1.50 | ms/batch 22.29 | loss  7.43 | ppl  1688.24
| epoch   1 | 12000/